In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
import dipy.reconst.sfm as sfm
import dipy.reconst.csdeconv as csd
import dipy.reconst.dti as dti
import dipy.core.gradients as grad

In [3]:
import utils
from model import Model, BiExponentialIsotropicModel

In [4]:
data = utils.read_data()

In [5]:
train_data = data['seen']['signal'][:, 0][::2]
train_bvals = data['seen']['bvals'][::2]
train_bvecs = data['seen']['bvecs'][::2]
train_delta = data['seen']['delta'][::2]
train_Delta = data['seen']['Delta'][::2]
train_te = data['seen']['TE'][::2]
train_g = data['seen']['g'][::2]

test_data = data['seen']['signal'][:, 0][1::2]
test_bvals = data['seen']['bvals'][1::2]
test_bvecs = data['seen']['bvecs'][1::2]
test_delta = data['seen']['delta'][1::2]
test_Delta = data['seen']['Delta'][1::2]
test_te = data['seen']['TE'][1::2]
test_g = data['seen']['g'][1::2]


#test_data = train_data
#test_bvals = train_bvals
#test_bvecs = train_bvecs
#test_delta = train_delta 
#test_Delta = train_Delta 
#test_te = train_te 
#test_g = train_g


In [6]:
train_gtab = grad.gradient_table(train_bvals, train_bvecs, big_delta=train_Delta, small_delta=train_delta)
test_gtab = grad.gradient_table(test_bvals, test_bvecs, big_delta=test_Delta, small_delta=test_delta)

In [7]:
train_s0 = train_data[train_gtab.b0s_mask]

In [ ]:
#model = sfm.SparseFascicleModel(train_gtab, isotropic=sfm.ExponentialIsotropicModel)
#model = dti.TensorModel(train_gtab)
model = Model(train_gtab, isotropic=BiExponentialIsotropicModel)#, alpha=0.000000001, l1_ratio=0.01)

In [ ]:
fit = model.fit(train_data, TE=train_te)

In [ ]:
predict= fit.predict(test_gtab, test_te)

In [ ]:
np.unique(train_Delta)

In [ ]:
plt.plot(train_delta, train_gtab.bvals, '.')

In [ ]:
fig, ax = plt.subplots(1)
for this_Delta in np.unique(train_Delta):
    idx = train_Delta==this_Delta
    ax.plot(train_gtab.bvals[idx]/this_Delta, train_data[idx], '.', label=this_Delta)

In [ ]:
fig, ax = plt.subplots(1)
for this_g in np.unique(train_g):
    idx = train_g==this_g
    ax.plot(train_gtab.bvals[idx], train_data[idx], '.', label=this_g)
plt.legend()

In [ ]:
plt.plot(test_gtab.bvals, test_data, '.')
plt.plot(test_gtab.bvals, predict, '.')

In [ ]:
plt.plot(test_gtab.bvals, test_data, '.')
plt.plot(test_gtab.bvals, predict, '.')

In [ ]:
utils.LSE(predict, test_data)

In [ ]:
np.unique(test_Delta).shape

In [ ]:
with sns.color_palette("hls", 30):
    fig, ax = plt.subplots(1)
    for this_Delta in np.unique(test_Delta):
        idx = test_Delta==this_Delta
        ax.plot(predict[idx], test_data[idx], '.', label=this_Delta)
        ax.invert_xaxis()
        ax.invert_yaxis()
        plt.plot([0, 450], [0, 450], 'k--')
        ax.loglog()
    plt.legend()

In [ ]:
import dipy.viz.projections as proj

In [ ]:
import dipy.data as dpd

In [ ]:
s = dpd.get_sphere()

In [ ]:
fit.beta.shape

In [ ]:
proj.sph_project(s.vertices.T, fit.beta, vmax=0.3)

In [ ]:
fit.te_params

In [ ]:
proj.sph_project(s.vertices.T, fit.odf(s), vmax=0.3)

In [ ]:
plt.plot(test_Delta, test_te, 'o')
plt.xlabel('$\Delta$')